In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#load and understand the data

df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\Codenera\Projects\Banking Loan Default Analysis\data\credit_risk_dataset.csv")
print("====Business Data Overview====")
print("Shape of dataset:",df.shape)
print(f"Total Loans : {len(df):,}")
print(f"Default rate: {(df['loan_status'].mean()*100):.1f}%")
print(f"Total loan Portfolio: ${df['loan_amnt'].sum():,}")

#Add business context column

np.random.seed(42)


#1.Branch Information (10 branches across regions)
branches = {
    'BR001': 'North','BR002' : 'North' , 'BR003' : 'South',
    'BR004' : 'South' , 'BR005' : 'East' , 'BR006' : 'East',
    'BR007' : 'West' , 'BR008' : 'West' , 'BR009' : 'Central' , 'BR010' : 'Central'
}

df['branch_id'] = np.random.choice(list(branches.keys()), size = len(df))
df['region'] = df['branch_id'].map(branches)

#2.Application timeline (2-year period)

dates = pd.date_range('2011-01-01','2022-12-13',freq='D')   #It creates a list of dates starting from 1 January 2011 to 13 December 2022, with a frequency of 1 day.
df['application_date'] = np.random.choice(dates,size = len(df))
df['application_month'] = df['application_date'].dt.to_period('M')
df['application_quarter'] = df['application_date'].dt.quarter
df['application_year'] = df['application_date'].dt.year

#3.customer segment

df['customer_segment'] = pd.cut(df['person_income'],
                                bins = [0,30000,60000,100000,np.inf],
                                labels = ['Low Income','Medium Income','High Income','Very High Income'])


#4 .Loan size categories

df['loan_size'] = pd.cut(df['loan_amnt'],
                        bins = [0,10000,25000,50000,np.inf],
                        labels = ['Small','Medium','Large','Very Large'])

print("\n=== BUSINESS CONTEXT ADDED ===")
print(f"Branches: {df['branch_id'].nunique()} branches across {df['region'].nunique()} regions")
print(f"Time Period :{df['application_date'].min()} to {df['application_date'].max()}")
print(f"Customer Segment: {df['customer_segment'].value_counts().to_dict()}")



====Business Data Overview====
Shape of dataset: (32581, 12)
Total Loans : 32,581
Default rate: 21.8%
Total loan Portfolio: $312,431,300

=== BUSINESS CONTEXT ADDED ===
Branches: 10 branches across 5 regions
Time Period :2011-01-01 00:00:00 to 2022-12-13 00:00:00
Customer Segment: {'Medium Income': 14210, 'High Income': 9648, 'Low Income': 4516, 'Very High Income': 4207}
Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length', 'branch_id',
       'region', 'application_date', 'application_month',
       'application_quarter', 'application_year', 'customer_segment',
       'loan_size'],
      dtype='object')


In [12]:
#Data quality assessment for business
print("=== DATA QUALITY REPORT ===")

#1. Missing values business impact

missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_report = pd.DataFrame({
    'Missing Count':missing_data,
    'Missing %' : missing_percent,
    'Business Impact' : ['HIGH' if x > 5 else 'MEDIUM' if x > 1 else 'LOW' for x in missing_percent]
})
print("Missing Values Analysis:")
print(missing_report[missing_report['Missing Count'] > 0])


# 2. Handle missing values with business logic
# Employment length - use median by income segment

df['person_emp_length'] = df.groupby('customer_segment')['person_emp_length'].transform(
    lambda x: x.fillna(x.median()))

#Interest rate - use median by loan grade (common industry practice)
df['loan_int_rate'] = df.groupby('loan_grade')['loan_int_rate'].transform(
    lambda x: x.fillna(x.median()))


# 3. Remove business outliers
print(f"\nRecords before outlier removal: {len(df)}")
df = df[df['person_age'] <= 80]  #Realistic age
df = df[df['person_income'] <= 500000]  #Realistic income
print(f"Records after outlier removal: {len(df)}")

print("\n=== DATA QUALITY STATUS ===")
print("All missing values handled with business-appropriate methods")
print(f"Final dataset : {len(df):,} loans | ${df['loan_amnt'].sum():,} portfolio")
df.to_csv(r"C:\Users\ASUS\OneDrive\Desktop\Codenera\Projects\Banking Loan Default Analysis\notebooks\cleaned_loan_data.csv", index=False)



=== DATA QUALITY REPORT ===
Missing Values Analysis:
Empty DataFrame
Columns: [Missing Count, Missing %, Business Impact]
Index: []

Records before outlier removal: 32522
Records after outlier removal: 32522

=== DATA QUALITY STATUS ===
All missing values handled with business-appropriate methods
Final dataset : 32,522 loans | $311,702,000 portfolio


C:\Users\ASUS\AppData\Local\Temp\ipykernel_13840\3342515978.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['person_emp_length'] = df.groupby('customer_segment')['person_emp_length'].transform(


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\ASUS\\OneDrive\\Desktop\\Codenera\\Projects\\Banking Loan Default Analysis\\notebooks\\cleaned_loan_data.csv'